In [4]:
(15.1 + 15.3 + 2.7)/(15.1 +43.6 + 15.3 +2.7)

0.43155149934810955

初始化，导包读数据

In [18]:
import pandas as pd
import numpy as np
output_path = "D:/temp_files/datasets/titanic/"
train_path = "D:/temp_files/datasets/titanic/train.csv"
test_path = "D:/temp_files/datasets/titanic/test.csv"
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [10]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [11]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


简单预处理

In [12]:
# 先把小于1的
train_data.loc[train_data.Age < 1, 'Age'] = 1
test_data.loc[test_data.Age < 1, 'Age'] = 1
# 使用平均年龄来填充年龄中nan值
train_data['Age'].fillna(train_data.Age.mean(), inplace=True)
test_data['Age'].fillna(test_data.Age.mean(), inplace=True)
test_data['Fare'].fillna(test_data.Fare.mean(), inplace=True)

In [13]:
# 观察Embarked值的种类
train_data.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [14]:
train_data['Embarked'].fillna('S', inplace=True)
test_data['Embarked'].fillna('S', inplace=True)

* 特征选择

    应该先检查在该领域中是否存在成熟的理论模型，若存在，则依据该理论模型来选取特征，否则，自行选取（在这个问题上）
    

In [15]:
# 特征选择
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = train_data[features]
y = train_data['Survived']
# from sklearn.model_selection import train_test_split
# train_X, validate_X,train_y, validate_y= train_test_split(X, y, random_state=222200801)
test_X = test_data.loc[:, features]

* 特征提取和缩放

    使用pd.get_dummies()来将特征独热编码， 使用minmaxscaler进行归一化

In [16]:
features_need_dummied: list[str] = ['Embarked','Sex']
features_need_scaled = ['Fare', 'Age']
# 独热编码
def implement_dummies(data: pd.DataFrame, features: list[str]) -> pd.DataFrame:
    res = pd.DataFrame()
    for feature in features:
        dummy = pd.get_dummies(data[feature])
        res = pd.concat([res, dummy], axis=1)
    data = data.drop(labels=features, axis=1)
    data = pd.concat([data, res], axis=1)
    return data

# 归一化
def implement_minmax_scaler(data: pd.DataFrame, features: list[str]) -> pd.DataFrame:
    from sklearn.preprocessing import MinMaxScaler
    res = pd.DataFrame()
    for feature in features:
        scaler = MinMaxScaler()
        scaled = scaler.fit_transform(pd.DataFrame(data.index + data[feature]))
        scaled = pd.DataFrame(scaled, columns=[feature])
        res = pd.concat([res, scaled], axis=1)
    data = data.drop(labels=features, axis=1)
    data = pd.concat([data, res], axis=1)
    return data


In [17]:
X = implement_dummies(X, features_need_dummied)
test_X = implement_dummies(test_X, features_need_dummied)
X = implement_minmax_scaler(X, features_need_scaled)
test_X = implement_minmax_scaler(test_X, features_need_scaled)

将X、test_X、y存入文件


In [ ]:
X.to_csv(output_path + "X" + ".csv",index=True)
test_X.to_csv(output_path + "test_X" + ".csv",index=True)
y.to_csv(output_path + "y" + ".csv",index=True)